In [1]:
import os
import numpy as np
from time import time
import datetime
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import label_binarize

### 1、Load Data

In [2]:
filepath ='/home/dell/Xinda/SVM/server/Audio/data_opensmile/eGeMAPs_6PNN.csv'
data = pd.read_csv(filepath)
x = data.iloc[:, :-1]  # 数据特征
y = data.iloc[:,-1]  # 标签

# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=423)

In [3]:
data

,audio_1,audio_2,audio_3,audio_4,audio_5,audio_6,audio_7,audio_8,audio_9,audio_10,...,audio_80,audio_81,audio_82,audio_83,audio_84,audio_85,audio_86,audio_87,audio_88,affection
0,32.11512,0.197363,31.85040,34.13453,35.71888,3.868471,264.92140,315.97390,341.18750,575.10880,...,-0.004767,0.052806,3.783784,1.381216,0.612000,0.409898,0.090000,0.068411,-42.88590,中性
1,30.23483,0.266454,21.13014,33.97304,36.63100,15.500860,184.26240,134.92030,55.78628,50.71881,...,-0.002601,0.034639,5.384615,1.181102,0.723333,0.203688,0.083333,0.049889,-42.34191,中性
2,28.92495,0.303522,17.47816,34.62748,36.72791,19.249740,370.87990,603.46200,151.41780,104.63920,...,-0.003773,0.058610,3.740649,2.798982,0.278182,0.335229,0.080000,0.064226,-41.42824,紧张
3,30.70091,0.278915,21.54842,33.88435,38.08253,16.534110,213.92850,154.79490,73.61297,55.43237,...,-0.003445,0.056165,3.571429,1.444043,0.586250,0.563958,0.097143,0.068183,-37.92779,紧张
4,28.40746,0.293685,17.24780,33.62654,35.42783,18.180030,287.73480,211.60730,106.33080,76.48589,...,-0.006646,0.094086,3.547672,3.363229,0.226667,0.237112,0.060833,0.041121,-41.92458,紧张
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12124,34.04372,0.131364,34.07095,35.34908,37.04111,2.970161,203.76630,65.19341,94.43509,69.82155,...,-0.001546,0.032072,3.726708,2.597403,0.325000,0.402772,0.040000,0.021213,-48.89381,紧张
12125,36.44316,0.129016,35.62368,36.97424,39.14269,3.519009,607.56050,516.65630,28.24421,15.10102,...,-0.012557,0.026442,2.879079,1.747573,0.324444,0.160493,0.290000,0.189435,-47.95839,紧张
12126,38.29095,0.123908,37.88574,39.26852,41.40741,3.521671,1372.95000,1351.52900,78.34052,53.36443,...,-0.016557,0.035311,4.232804,3.278689,0.198333,0.113639,0.130000,0.144568,-46.84602,紧张
12127,35.17179,0.165482,35.46397,36.80436,39.36259,3.898613,34.33191,51.26659,121.40470,115.70860,...,-0.017121,0.055657,3.703704,2.732240,0.294000,0.240882,0.086667,0.054365,-46.34153,紧张


In [4]:
x_test

,audio_1,audio_2,audio_3,audio_4,audio_5,audio_6,audio_7,audio_8,audio_9,audio_10,...,audio_79,audio_80,audio_81,audio_82,audio_83,audio_84,audio_85,audio_86,audio_87,audio_88
11720,18.60810,0.184257,16.34262,18.98980,21.67671,5.334091,195.13290,0.000000,-300.95910,0.000000,...,-0.007897,-0.006654,0.019763,5.882353,0.900901,0.0700,0.000000,0.505000,0.105000,-60.43456
1250,23.97624,0.054210,22.92682,23.76748,24.88933,1.962509,31.76878,10.296240,14.72799,5.349873,...,-0.013239,-0.006794,0.021266,2.484472,1.290323,0.4200,0.200000,0.335000,0.315000,-47.84021
11051,25.18750,0.130377,23.51089,24.13974,27.74739,4.236500,394.07200,672.073200,106.33980,85.428730,...,0.017163,-0.014190,0.056195,4.093567,2.469136,0.3200,0.209643,0.083333,0.063421,-45.04489
1670,33.97573,0.200187,30.33104,33.38041,40.40098,10.069940,262.82960,168.434100,74.81174,56.897320,...,-0.000150,-0.015716,0.029320,3.571429,2.985075,0.2025,0.166339,0.225000,0.205000,-46.96961
4946,22.54483,0.095509,21.73226,22.96850,23.92953,2.197277,68.37223,63.514550,17.99337,6.780934,...,-0.000917,-0.005552,0.020177,1.718213,0.701754,0.5150,0.105000,0.590000,0.359537,-52.83109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,23.79777,0.123161,21.88373,24.31328,26.16268,4.278952,27.50193,9.005311,45.88934,15.006700,...,-0.010642,0.000313,0.014809,3.875969,0.840336,0.9000,0.000000,0.270000,0.000000,-45.81554
157,32.00220,0.210084,28.53207,34.63807,36.21652,7.684454,216.06290,165.294800,173.79080,282.699000,...,0.036204,-0.000684,0.092524,4.456825,2.832861,0.2470,0.208329,0.076364,0.058196,-41.58880
1606,23.78821,0.062657,23.44209,23.79415,24.79837,1.356285,165.88750,168.057200,89.12145,77.761590,...,0.004894,-0.014419,0.035215,3.765691,2.575107,0.2650,0.158193,0.087143,0.094976,-46.56576
777,32.45066,0.049662,31.80017,32.65693,33.33179,1.531612,242.94230,483.179600,39.96987,35.743830,...,0.011444,-0.019080,0.032474,7.112971,2.991453,0.1700,0.087505,0.127500,0.150976,-50.74685


In [6]:
# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# Y 将标签编码
encoder = LabelEncoder().fit(y_train) # #训练LabelEncoder, 把y_train中的类别编码为0，1，2，3，4，5
y = encoder.transform(y_train)
print("class = ", y)
y_train = pd.DataFrame(encoder.transform(y_train)) # 使用训练好的LabelEncoder对源数据进行编码
y_test = pd.DataFrame(encoder.transform(y_test))
print("y_test = ", y_test[0:10])

# 标签降维度
y_train = y_train.iloc[:,0].ravel()
y_test = y_test.iloc[:,0].ravel()
print("y.shape=", y_train.shape)

# X标准化
scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train)  # 标准化
x_test_std = scaler.fit_transform(x_test)

class =  [2 5 0 ... 2 0 0]
y_test =     0
0  1
1  0
2  0
3  3
4  0
5  4
6  5
7  4
8  3
9  4
y.shape= (8490,)


## Parameters for C and Gamma

In [19]:
time0 = time()
clf = OneVsRestClassifier(
        SVC(kernel = 'rbf', 
        gamma =  0.04888888888888889,
            C = 1.13333333333,
        degree=1, 
        probability=True,
        cache_size=5000, 
        class_weight='balanced'))
clf.fit(x_train_std, y_train)
    
y_test_prediction = clf.predict(x_test_std)
# accuracy
accuracy = accuracy_score(y_test,y_test_prediction)
print("accuracy = ", accuracy)
# F1-score
f1 = f1_score(y_test,y_test_prediction, average="weighted")
print("f1 score = ", f1)
    
# AUC
y_test_binary = label_binarize(y_test, classes=list(range(6))) # 转化为one-hot
result = clf.decision_function(x_test_std)
auc = roc_auc_score(y_test_binary, result, average = 'micro') # 多类分类下，要用概率值（形式二） ，加参数 average='micro'  （不能用ont-hot (形式三) ）
print("AUC = ", auc)

print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
print()

accuracy =  0.5229458642484199
f1 score =  0.506050599244661
AUC =  0.8311931269450423
19:43:127262



## For decision level fusion

In [20]:
result = clf.predict_proba(x_test_std)

In [22]:
df = pd.DataFrame(result)
df.to_csv("categorical01_opensmile_6pnn.csv")

In [12]:
y_test_prediction

array([0, 0, 4, ..., 0, 2, 0])

In [13]:
y_test

array([1, 0, 0, ..., 4, 2, 3])

In [14]:
df = pd.DataFrame(data={"opensmile_prediction_p": y_test_prediction, "opensmile_groundtruth_p": y_test.tolist()})
df.to_csv("eval_opensmile_6pnn.csv")
print("save success!")
print("ok")

save success!
ok


# Gamma -1

In [ ]:
import tqdm
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
accuracy_list = []
f1_list = []
auc_list = []

gamma_range = np.logspace(-10, 1, 10, base=2) # 返回13个数字，底是2
print("gamma_rang:", gamma_range)
count=1
for gamma_item in gamma_range:
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = OneVsRestClassifier(
        SVC(kernel = 'rbf', 
        gamma = gamma_item,
        degree=1, 
        cache_size=5000, 
        class_weight='balanced'))
    clf.fit(x_train_std, y_train)
    
    y_test_prediction = clf.predict(x_test_std)
    # accuracy
    accuracy = accuracy_score(y_test,y_test_prediction)
    accuracy_list.append(accuracy)
    print("accuracy = ", accuracy)
    # F1-score
    f1 = f1_score(y_test,y_test_prediction, average="weighted")
    print("f1 score = ", f1)
    f1_list.append(f1)
    # AUC
    y_test_binary = label_binarize(y_test, classes=list(range(6))) # 转化为one-hot
    result = clf.decision_function(x_test_std)
    auc = roc_auc_score(y_test_binary, result, average = 'micro') # 多类分类下，要用概率值（形式二） ，加参数 average='micro'  （不能用ont-hot (形式三) ）
    print("AUC = ", auc)
    auc_list.append(auc)
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()

print("Gamma = ", gamma_range[accuracy_list.index(max(accuracy_list))])
print("Accuracy = ", max(accuracy_list))
print("F1-score = ", f1_list[accuracy_list.index(max(accuracy_list))])
print("AUC-score = ", auc_list[accuracy_list.index(max(accuracy_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
accuracy =  0.3352569387194284
f1 score =  0.3344342936088522
AUC =  0.6897244976919076
06:22:937307

Start-2, gamma=0.0022783770304221013


##  C

In [ ]:
import tqdm
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
accuracy_list = []
f1_list = []
auc_list = []

c_range = np.linspace(0.01,30,50)
print("c_range:",c_range)
print()
count=1
for c_item in c_range:
    time0 = time()
    print("Start-{0}, c_item={1}".format(count, c_item))
    count = count+1
    clf = OneVsRestClassifier(
        SVC(kernel = 'rbf', 
        gamma = 0.06750373368076916,
        C = c_item,
        degree=1, 
        cache_size=5000, 
        class_weight='balanced'))
    clf.fit(x_train_std, y_train)
    
    y_test_prediction = clf.predict(x_test_std)
    # accuracy
    accuracy = accuracy_score(y_test,y_test_prediction)
    accuracy_list.append(accuracy)
    print("accuracy = ", accuracy)
    # F1-score
    f1 = f1_score(y_test,y_test_prediction, average="weighted")
    print("f1 score = ", f1)
    f1_list.append(f1)
    # AUC
    y_test_binary = label_binarize(y_test, classes=list(range(6))) # 转化为one-hot
    result = clf.decision_function(x_test_std)
    auc = roc_auc_score(y_test_binary, result, average = 'micro') # 多类分类下，要用概率值（形式二） ，加参数 average='micro'  （不能用ont-hot (形式三) ）
    print("AUC = ", auc)
    auc_list.append(auc)
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print(max(accuracy_list), gamma_range[accuracy_list.index(max(accuracy_list))])
print("F1-score = ", f1_list[accuracy_list.index(max(accuracy_list))])
print("AUC-score = ", auc_list[accuracy_list.index(max(accuracy_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

c_range: [1.00000000e-02 6.22040816e-01 1.23408163e+00 1.84612245e+00
 2.45816327e+00 3.07020408e+00 3.68224490e+00 4.29428571e+00
 4.90632653e+00 5.51836735e+00 6.13040816e+00 6.74244898e+00
 7.35448980e+00 7.96653061e+00 8.57857143e+00 9.19061224e+00
 9.80265306e+00 1.04146939e+01 1.10267347e+01 1.16387755e+01
 1.22508163e+01 1.28628571e+01 1.34748980e+01 1.40869388e+01
 1.46989796e+01 1.53110204e+01 1.59230612e+01 1.65351020e+01
 1.71471429e+01 1.77591837e+01 1.83712245e+01 1.89832653e+01
 1.95953061e+01 2.02073469e+01 2.08193878e+01 2.14314286e+01
 2.20434694e+01 2.26555102e+01 2.32675510e+01 2.38795918e+01
 2.44916327e+01 2.51036735e+01 2.57157143e+01 2.63277551e+01
 2.69397959e+01 2.75518367e+01 2.81638776e+01 2.87759184e+01
 2.93879592e+01 3.00000000e+01]

Start-1, c_item=0.01
accuracy =  0.25309150865622426
f1 score =  0.14567839176033887
AUC =  0.7122131988795619
02:47:913557


Start-2, c_item=0.6220408163265306
accuracy =  0.4932673811486672
f1 score =  0.46817848286988867
AU

### C 二次调节

In [4]:
c_range = np.linspace(0.6,1.8,10)
print(c_range)

[0.6        0.73333333 0.86666667 1.         1.13333333 1.26666667
 1.4        1.53333333 1.66666667 1.8       ]


In [5]:
import tqdm
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
accuracy_list = []
f1_list = []
auc_list = []

c_range = np.linspace(0.6,1.8,10)
print("c_range:",c_range)
print()
count=1
for c_item in c_range:
    time0 = time()
    print("Start-{0}, c_item={1}".format(count, c_item))
    count = count+1
    clf = OneVsRestClassifier(
        SVC(kernel = 'rbf', 
        gamma = 0.06750373368076916,
        C = c_item,
        degree=1, 
        cache_size=5000, 
        class_weight='balanced'))
    clf.fit(x_train_std, y_train)
    
    y_test_prediction = clf.predict(x_test_std)
    # accuracy
    accuracy = accuracy_score(y_test,y_test_prediction)
    accuracy_list.append(accuracy)
    print("accuracy = ", accuracy)
    # F1-score
    f1 = f1_score(y_test,y_test_prediction, average="weighted")
    print("f1 score = ", f1)
    f1_list.append(f1)
    # AUC
    y_test_binary = label_binarize(y_test, classes=list(range(6))) # 转化为one-hot
    result = clf.decision_function(x_test_std)
    auc = roc_auc_score(y_test_binary, result, average = 'micro') # 多类分类下，要用概率值（形式二） ，加参数 average='micro'  （不能用ont-hot (形式三) ）
    print("AUC = ", auc)
    auc_list.append(auc)
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print(max(accuracy_list), gamma_range[accuracy_list.index(max(accuracy_list))])
print("F1-score = ", f1_list[accuracy_list.index(max(accuracy_list))])
print("AUC-score = ", auc_list[accuracy_list.index(max(accuracy_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

c_range: [0.6        0.73333333 0.86666667 1.         1.13333333 1.26666667
 1.4        1.53333333 1.66666667 1.8       ]

Start-1, c_item=0.6
accuracy =  0.49381698268755153
f1 score =  0.46885771500224366
AUC =  0.8233474403769551
02:24:824027


Start-2, c_item=0.7333333333333334
accuracy =  0.5023358065402583
f1 score =  0.4767022390485015
AUC =  0.8270113298114432
02:24:479149


Start-3, c_item=0.8666666666666667
accuracy =  0.5072822203902171
f1 score =  0.4820506026439221
AUC =  0.8287848557665984
02:24:369366


Start-4, c_item=1.0
accuracy =  0.5111294311624073
f1 score =  0.48635222877203715
AUC =  0.8297233921455309
02:21:274839


Start-5, c_item=1.1333333333333333
accuracy =  0.512503435009618
f1 score =  0.4876587035817744
AUC =  0.83017415904659
02:31:511332


Start-6, c_item=1.2666666666666666
accuracy =  0.5122286342401758
f1 score =  0.48792732319232807
AUC =  0.8303139683745772
02:29:572412


Start-7, c_item=1.4000000000000001
accuracy =  0.5122286342401758
f1 score =  

NameError: name 'gamma_range' is not defined

### Gamma 2次调试

In [7]:
import tqdm
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
accuracy_list = []
f1_list = []
auc_list = []
gamma_range = np.linspace(0.02,0.15,10) # 
for gamma_item in gamma_range:
    count=1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count+1
    clf = OneVsRestClassifier(
        SVC(kernel = 'rbf', 
        gamma = gamma_item,
        C = 1.1333333333333333,
        degree=1, 
        cache_size=5000, 
        class_weight='balanced'))
    clf.fit(x_train_std, y_train)
    
    y_test_prediction = clf.predict(x_test_std)
    # accuracy
    accuracy = accuracy_score(y_test,y_test_prediction)
    accuracy_list.append(accuracy)
    print("accuracy = ", accuracy)
    # F1-score
    f1 = f1_score(y_test,y_test_prediction, average="weighted")
    print("f1 score = ", f1)
    f1_list.append(f1)
    # AUC
    y_test_binary = label_binarize(y_test, classes=list(range(6))) # 转化为one-hot
    result = clf.decision_function(x_test_std)
    auc = roc_auc_score(y_test_binary, result, average = 'micro') # 多类分类下，要用概率值（形式二） ，加参数 average='micro'  （不能用ont-hot (形式三) ）
    print("AUC = ", auc)
    auc_list.append(auc)
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))
    print()
    print()

print(max(accuracy_list), gamma_range[accuracy_list.index(max(accuracy_list))])
print("F1-score = ", f1_list[accuracy_list.index(max(accuracy_list))])
print("AUC-score = ", auc_list[accuracy_list.index(max(accuracy_list))]) 
print(datetime.datetime.fromtimestamp(time()-times_all).strftime("%M:%S:%f"))

Start-1, gamma=0.02
accuracy =  0.4943665842264358
f1 score =  0.490974017405516
AUC =  0.8118934286519713
01:51:997030


Start-1, gamma=0.034444444444444444
accuracy =  0.5144270403957131
f1 score =  0.502910584851038
AUC =  0.8273705342137531
01:26:120539


Start-1, gamma=0.04888888888888889
accuracy =  0.5229458642484199
f1 score =  0.506050599244661
AUC =  0.8311931269450423
01:29:544614


Start-1, gamma=0.06333333333333334
accuracy =  0.5133278373179445
f1 score =  0.4898461411152601
AUC =  0.8307813562290176
01:40:543625


Start-1, gamma=0.07777777777777778
accuracy =  0.5048090134652377
f1 score =  0.4763304304475581
AUC =  0.8278780660882636
01:50:261940


Start-1, gamma=0.09222222222222223
accuracy =  0.49381698268755153
f1 score =  0.4592420208617849
AUC =  0.8230942596845371
02:09:293421


Start-1, gamma=0.10666666666666667
accuracy =  0.47348172574883207
f1 score =  0.4310844503007552
AUC =  0.8171394803071155
02:12:504225


Start-1, gamma=0.12111111111111113
accuracy =  0.